In [1]:
from pyspark import SparkConf, SparkContext
import re

In [2]:
# Create a SparkContext
conf = SparkConf().setMaster("local").setAppName("PopularWikipediaPages")
sc = SparkContext(conf=conf)

In [3]:
def parseLine(line):
    try:
        fields = line.split()
        project = fields[0]
        title = fields[1]
        hits = int(fields[2])
        size = int(fields[3])
        return (project, title, hits, size)
    except Exception as e:
        print("Error parsing line:", line)
        print("Exception:", e)
        return None

# Read the data from the file using parseLine function
lines = sc.textFile("pagecounts-20160101-000000_parsed.out")
rdd = lines.map(parseLine).filter(lambda x: x is not None)


In [4]:
# Compute min, max, and average page sizes
minSize = rdd.map(lambda x: x[3]).min()
maxSize = rdd.map(lambda x: x[3]).max()
avgSize = rdd.map(lambda x: x[3]).mean()

print("Min page size: {}, Max page size: {}, Average page size: {}".format(minSize, maxSize, avgSize))


Min page size: 0, Max page size: 141180155987, Average page size: 132215.79814237214


In [5]:
# # Count page titles that start with "The"
# the_titles_count = rdd.filter(lambda x: x[1].startswith("The")).count()

# Count page titles that start with "The" and are not part of the English project
english_the_titles_count = rdd.filter(lambda x: x[0] != "en" and x[1].startswith("The")).count()
# print("The titles count: {}, English The titles count: {}".format(the_titles_count, english_the_titles_count))

print("English The titles count: {}".format(english_the_titles_count))

English The titles count: 10292


In [6]:
def preprocess_title(title):
    # Lowercase the title
    title = title.lower()
    # Remove non-alphanumeric characters an
    title = re.sub(r'[^a-z0-9_]', '', title)
    return title

# Preprocess page titles
preprocessed_titles = rdd.map(lambda x: preprocess_title(x[1]))
print(preprocessed_titles.take(5))

# Split titles into terms and flatten
terms = preprocessed_titles.flatMap(lambda title: title.split("_"))
print(terms.take(5))

# Count unique terms
unique_terms_count = terms.distinct().count()

print("Number of unique terms appearing in the page titles:", unique_terms_count)


# re.sub(): This function is used for performing substitutions based on regular expressions. It takes three main arguments:
# The first argument is the regular expression pattern to search for.
# The second argument is the replacement string, which will replace the matched pattern.
# The third argument is the string on which the operation is performed.
# r'[^a-zA-Z0-9_]': This regular expression pattern matches any character that is not alphanumeric (a-z, A-Z, 0-9) or an underscore _.
# '': This is the replacement string, which is an empty string. It means that any character that matches the pattern will be removed (replaced with nothing).
# 'title: This is the string on which the substitution operation is performed. In this case, it's the page title.
# So, the re.sub() function will remove all characters from the page title that are not alphanumeric or underscores.

# ["hello world", "foo bar", "baz"]
# map
# ["hello", "world"]
# ["foo", "bar"]
# ["baz"]
# flatMap
# ["hello", "world", "foo", "bar", "baz"]

['271_ac', 'categoryuser_th', 'chiron_elias_krase', 'dassault_rafaele', 'edesv']
['271', 'ac', 'categoryuser', 'th', 'chiron']
Number of unique terms appearing in the page titles: 1688528


In [7]:
# Extract each title and the number of times it was repeated
title_counts = rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y)

for title, count in title_counts.take(10):
    print("Title: {}, Count: {}".format(title, count))

Title: E.Desv, Count: 6
Title: Special:Contributions/5.232.61.79, Count: 1
Title: Special:ListFiles/Nyttend, Count: 1
Title: Special:WhatLinksHere/Main_Page, Count: 8
Title: Time_Inc, Count: 4
Title: Special:Contributions/MBisanz, Count: 1
Title: Special:UserLogin, Count: 13
Title: Acanthophorus_serraticornis, Count: 6
Title: Allen_R._Schindler,_Jr, Count: 6
Title: Annales._Histoire,_Sciences_Sociales/en/Annales_d, Count: 4


In [8]:
# Combine between data of pages with the same title and save each pair of pages data in order to display them
combined_titles = rdd.map(lambda x: (x[1], (x[0], x[2], x[3]))).groupByKey().filter(lambda x: len(x[1]) > 1).mapValues(list)
# combined_titles = rdd.map(lambda x: (x[1], (x[0], x[2], x[3]))).groupByKey().filter(lambda x: len(x[1]) > 1).mapValues(list)

print("Combined titles:")
for title, data in combined_titles.take(10):
    print("Title: {}, Data: {}".format(title, data))

# mapValues(list) to convert the iterator of values into a list. This will give you each title paired with a list of its corresponding data tuples.

Combined titles:
Title: E.Desv, Data: [('aa', 1, 4662), ('arc', 1, 5210), ('ast', 1, 4825), ('fiu-vro', 1, 5237), ('fr', 1, 7057), ('ik', 1, 4548)]
Title: Special:WhatLinksHere/Main_Page, Data: [('aa', 1, 5556), ('commons.m', 2, 15231), ('en', 5, 101406), ('en.s', 1, 8597), ('en.voy', 1, 8550), ('meta.m', 1, 11529), ('outreach.m', 1, 5698), ('simple', 3, 32145)]
Title: Time_Inc, Data: [('aa', 1, 4672), ('af', 1, 6182), ('ha', 1, 4842), ('na', 1, 4923)]
Title: Special:UserLogin, Data: [('aa.d', 1, 4899), ('commons.m', 30, 181938), ('en', 44198, 718770014), ('en.q', 4, 34449), ('incubator.m', 1, 5221), ('m.f', 13, 58547), ('m.w', 3, 12523), ('m.wd', 2, 8696), ('meta.m', 1, 5311), ('simple', 2, 9960), ('ss', 1, 5052), ('www.w', 5, 49952), ('www.wd', 1, 4989)]
Title: Acanthophorus_serraticornis, Data: [('ab', 1, 5942), ('ckb', 1, 5825), ('csb', 1, 5480), ('de', 1, 5510), ('dsb', 1, 5429), ('fj', 1, 4693)]
Title: Allen_R._Schindler,_Jr, Data: [('ab', 1, 5957), ('ckb', 1, 5840), ('fj', 1, 47

In [9]:
# create a document includes all the results of each query
with open("results.txt", "w", encoding="utf-8") as f:
    f.write("Min page size: {}\n".format(minSize))
    f.write("Max page size: {}\n".format(maxSize))
    f.write("Average page size: {}\n".format(avgSize))
    f.write("English The titles count: {}\n".format(english_the_titles_count))
    f.write("Number of unique terms appearing in the page titles: {}\n".format(unique_terms_count))
    f.write("\n")
    f.write("Title Counts:\n")
    for title, count in title_counts.collect():
        f.write("{}: {}\n".format(title, count))
    f.write("\n")
    f.write("Combined Titles:\n")
    for title, data_list in combined_titles.collect():
        f.write("{}:\n".format(title))
        for data in data_list:
            f.write("{}\n".format(data))


In [10]:
# Stop the existing SparkContext
sc.stop()  
